In [ ]:
! pip install pymysql google-search-results

In [29]:
import os
os.environ['GOOGLE_API_KEY']='GOOGLE_API_KEY'
os.environ['SERPAPI_API_KEY']  ='SERPAPI_API_KEY'

In [14]:
from langchain.agents import load_tools,initialize_agent,AgentType
from langchain_google_genai import ChatGoogleGenerativeAI

In [11]:
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')
tools=load_tools(['serpapi','llm-math'],llm=llm)

In [19]:
agent = initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)
agent.run("Which term is Narendra currently serving? Subtract that term number from his age ?")



> Entering new AgentExecutor chain...
Thought: I need to find Narendra Modi's current term number and his age to answer this question.  I can use a search engine for this information.

Action: Search
Action Input: "Narendra Modi age" and "Narendra Modi current term"
Observation: [' entity_type: related_questions.', 'Narendra Damodardas Modi. (1950-09-17) 17 September 1950 (age 74) Vadnagar, Bombay State, India (present-day Gujarat) · Bharatiya Janata Party.', 'In full: Narendra Damodardas Modi ; Born: September 17, 1950, Vadnagar, India (age 74) ; Title / Office: prime minister (2014-), India.', "Shri Narendra Modi was sworn-in as India's Prime Minister for the third time on 9th June 2024, following another decisive victory in the 2024 Parliamentary ...", 'Narendra Modi, the present serving Prime Minister of India, was born on September 17, 1950. At present, in 2024, Narendra Modi age is 74. Throughout his ...', 'Term length ... Narasimha Rao, Atal Bihari Vajpayee, Manmohan Singh, an

'71'

In [20]:
agent=initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age as compared to him ?")



> Entering new AgentExecutor chain...
Thought: I need to find out who Leo DiCaprio's current girlfriend is and their ages to compare them.  I can use a search engine for this.

Action: Search
Action Input: "Leo DiCaprio girlfriend age"
Observation: [{'title': "Leonardo DiCaprio's topless girlfriend Vittoria Ceretti, 26, reveals intimate details on relationship with act", 'link': 'https://www.dailymail.co.uk/tvshowbiz/article-14531811/leonardo-dicaprio-girlfriend-vittoria-ceretti-reveals-intimate-details-relationship.html', 'source': 'Daily Mail', 'date': '2 days ago', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRvDUemGNodVJMLUvoqb6953cignNKLTsFQ39lz3w2yuCdvizsO2uxTwkg&usqp=CAI&s'}, {'title': "Leonardo DiCaprio's Girlfriend Vittoria Ceretti Breaks Silence on Romance - E! Online", 'link': 'https://www.eonline.com/news/1415186/leonardo-dicaprios-girlfriend-vittoria-ceretti-breaks-silence-on-romance', 'source': 'E! Online', 'date': '2 days ago', 'thumbnail': 'http

"Leonardo DiCaprio's girlfriend, Vittoria Ceretti, is 26 years old.  This is 22 years younger than Leonardo DiCaprio, who is 48 years old."

# LangChain agent which can talk with SQL DataBase

DataBase - https://www.mysqltutorial.org/getting-started-with-mysql/mysql-sample-database/

In [21]:
from langchain.agents import create_sql_agent,AgentExecutor
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
db_user='root'
db_password='PASSWORD'
db_host='localhost:3306'
db_name='classicmodels'
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

In [24]:
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')
toolkit=SQLDatabaseToolkit(db=db,llm=llm)
agent_executer=create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

Describing Table

In [25]:
agent_executer.run('Describe the Order related table and how they are related')



> Entering new SQL Agent Executor chain...
Thought: I should first list all tables to see if there are any related to "Order".
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsThought:The tables `orders`, `orderdetails`, `customers`, and `products` seem relevant to the question. I will query their schemas.
Action: sql_db_schema
Action Input: orders,orderdetails,customers,products
CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`addressLine1` VARCHAR(50) NOT NULL, 
	`addressLine2` VARCHAR(50), 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50), 
	`postalCode` VARCHAR(15), 
	country VARCHAR(50) NOT NULL, 
	`salesRepEmployeeNumber` INTEGER, 
	`creditLimit` DECIMAL(10, 2), 
	PRIMARY KEY (`customerNumber`), 
	CONSTRAINT customers_ibfk_1 FOREIGN KEY(

'The database contains four tables related to orders: `orders`, `orderdetails`, `customers`, and `products`.  The `orders` table contains general order information and links to the `customers` table via a foreign key (`customerNumber`). The `orderdetails` table links to both the `orders` table (via `orderNumber`) and the `products` table (via `productCode`), creating a many-to-many relationship between orders and products.  The `customers` table stores customer details, and the `products` table holds product information.'

In [27]:
agent_executer.run('Describe the OrderDetails table ')



> Entering new SQL Agent Executor chain...
Thought: I need to get a list of tables first.
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsI need to get the schema for the OrderDetails table.
Action: sql_db_schema
Action Input: orderdetails
CREATE TABLE orderdetails (
	`orderNumber` INTEGER NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER NOT NULL, 
	`priceEach` DECIMAL(10, 2) NOT NULL, 
	`orderLineNumber` SMALLINT NOT NULL, 
	PRIMARY KEY (`orderNumber`, `productCode`), 
	CONSTRAINT orderdetails_ibfk_1 FOREIGN KEY(`orderNumber`) REFERENCES orders (`orderNumber`), 
	CONSTRAINT orderdetails_ibfk_2 FOREIGN KEY(`productCode`) REFERENCES products (`productCode`)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from orderdetails table:
orderNumber	productCode	quantityOrdered	priceEach	orderLineNumber
10100	S18_1749	30	136.00	3
10100	S18_2248	50	55.09	2
10100	S

'The OrderDetails table has the following columns: orderNumber (integer, not null, primary key), productCode (varchar(15), not null, primary key), quantityOrdered (integer, not null), priceEach (decimal(10,2), not null), orderLineNumber (smallint, not null).  It also has foreign key constraints referencing the orders and products tables.  Each row represents a specific product within an order, including the quantity ordered and price.'

In [28]:
agent_executer.run('Find the top 5 products with highest total sales revenue')



> Entering new SQL Agent Executor chain...
Thought: I need to know what tables are available in the database.
Action: sql_db_list_tables
Action Input: ""customers, employees, offices, orderdetails, orders, payments, productlines, productsI need to find information about products and sales.  I'll examine the schemas for `products`, `orderdetails`, and `orders` tables.
Action: sql_db_schema
Action Input: products,orderdetails,orders
CREATE TABLE orderdetails (
	`orderNumber` INTEGER NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER NOT NULL, 
	`priceEach` DECIMAL(10, 2) NOT NULL, 
	`orderLineNumber` SMALLINT NOT NULL, 
	PRIMARY KEY (`orderNumber`, `productCode`), 
	CONSTRAINT orderdetails_ibfk_1 FOREIGN KEY(`orderNumber`) REFERENCES orders (`orderNumber`), 
	CONSTRAINT orderdetails_ibfk_2 FOREIGN KEY(`productCode`) REFERENCES products (`productCode`)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from orderdetails table:
orderNumb

"I don't know"